In [1]:
import numpy as np
import opt_einsum as oe
from copy import deepcopy
import itertools
from scipy import integrate
import pickle as pk
import copy
import math
import scipy

import torch
torch.set_default_dtype(torch.float64)

import os
import sys
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy'))
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy/tnpy'))

import tnpy as tp

SyntaxError: invalid syntax (quantum_boson_square.py, line 131)

In [4]:
%reload_ext autoreload
%autoreload all

t = torch.randn(4, 3)
print(t)

u, s, v = torch.linalg.svd(t, full_matrices=False)
print(u.shape, s.shape, v.shape)
print(u)
print(s)
print(v)

print(torch.linalg.norm((u @ s.diag() @ v)-t))

u, s, v = tp.linalg.svd(t)
print(u.shape, s.shape, v.shape)
print(u)
print(s)
print(v)

print(torch.linalg.norm((u @ s.diag() @ v)-t))

tensor([[-1.19889,  0.02258,  1.22681],
        [ 0.64739, -0.18390, -1.05985],
        [-1.14678, -1.35689, -0.24403],
        [-0.01183, -1.39781,  0.28115]])
torch.Size([4, 3]) torch.Size([3]) torch.Size([3, 3])
tensor([[ 0.58773,  0.53913, -0.04522],
        [-0.35289, -0.49005, -0.21365],
        [ 0.61638, -0.50100, -0.57488],
        [ 0.38746, -0.46710,  0.78855]])
tensor([2.34603, 1.89191, 0.82754])
tensor([[-0.70098, -0.55404,  0.44908],
        [-0.20273,  0.75850,  0.61933],
        [ 0.68376, -0.34310,  0.64401]])
tensor(1.13574e-15)
torch.Size([4, 3]) torch.Size([3]) torch.Size([3, 3])
tensor([[ 0.58773,  0.53913, -0.04522],
        [-0.35289, -0.49005, -0.21365],
        [ 0.61638, -0.50100, -0.57488],
        [ 0.38746, -0.46710,  0.78855]])
tensor([2.34603, 1.89191, 0.82754])
tensor([[-0.70098, -0.55404,  0.44908],
        [-0.20273,  0.75850,  0.61933],
        [ 0.68376, -0.34310,  0.64401]])
tensor(1.13574e-15)


In [41]:
# Heisenberg model
# exact GS energy: E=-0.66944
%reload_ext autoreload
%autoreload all

chi = 3
nx, ny = 2, 2


tps = tp.tps.SquareTPS.randn(nx=nx, ny=ny, chi=chi, dtype=torch.float64)
print(tps.coords, tps.nx, tps.ny)

xyz = tp.models.SquareXYZ(Jx=1.0, Jy=1.0, Jz=1.0, dtype=torch.float64)
ham_xyz = xyz.twobody_ham()
print(ham_xyz)
teo = xyz.twobody_img_time_evo(delta=0.1)

print(xyz.sx @ xyz.sx)
print(xyz.sy @ xyz.sy)
print(xyz.sz @ xyz.sz)

((0, 0), (1, 0), (0, 1), (1, 1)) 2 2
tensor([[[[ 0.25000,  0.00000],
          [ 0.00000, -0.25000]],

         [[ 0.00000,  0.00000],
          [ 0.50000,  0.00000]]],


        [[[ 0.00000,  0.50000],
          [ 0.00000,  0.00000]],

         [[-0.25000,  0.00000],
          [ 0.00000,  0.25000]]]])
tensor([[0.25000, 0.00000],
        [0.00000, 0.25000]])
tensor([[0.25000+0.j, 0.00000+0.j],
        [0.00000+0.j, 0.25000+0.j]])
tensor([[0.25000, 0.00000],
        [0.00000, 0.25000]])


In [42]:
deltas = [1E-2, 1E-3, 1E-4, 1E-5, 1E-6]
nums = [1000]*len(deltas)

counter = 0
for d, n in zip(deltas, nums):

    teo = xyz.twobody_img_time_evo(delta=d)
    u, s, v = tp.linalg.tsvd(teo, group_dims=((0, 2), (1, 3)), svd_dims=(1, 0))
    ss = torch.sqrt(s).diag().to(xyz.dtype)
    us = torch.einsum('abc,bB->aBc', u, ss)
    sv = torch.einsum('Aa,abc->Abc', ss, v)
    te_mpo = us, sv

    for l in range(n):
        old_lts = tps.link_tensors()
        tps.simple_update_proj(te_mpo)
        counter += 1

        if l % 500 == 0:
            new_lts = tps.link_tensors()
            diff = 0.0
            for key, value in new_lts.items():
                diff += torch.linalg.norm(value[0]-old_lts[key][0])
                diff += torch.linalg.norm(value[1]-old_lts[key][1])
            print(d, l, diff)

            beta_meas = tps.beta_twobody_measure(ham_xyz)
            print(beta_meas, torch.mean(beta_meas))

0.01 0 tensor(5.47324)
tensor([0.14633, 0.06059, 0.15497, 0.12867, 0.09095, 0.17240, 0.12223, 0.15201]) tensor(0.12852)
0.01 500 tensor(0.00471)
tensor([-0.25517, -0.22791, -0.25208, -0.25418, -0.25573, -0.23662, -0.25548,
        -0.25400]) tensor(-0.24890)
0.001 0 tensor(0.00589)
tensor([-0.25607, -0.25614, -0.25641, -0.25622, -0.25619, -0.25661, -0.25654,
        -0.25668]) tensor(-0.25636)
0.001 500 tensor(8.42438e-06)
tensor([-0.25631, -0.25632, -0.25636, -0.25634, -0.25634, -0.25640, -0.25640,
        -0.25642]) tensor(-0.25636)
0.0001 0 tensor(0.00058)
tensor([-0.25634, -0.25635, -0.25638, -0.25636, -0.25636, -0.25640, -0.25639,
        -0.25641]) tensor(-0.25637)
0.0001 500 tensor(5.74856e-07)
tensor([-0.25635, -0.25635, -0.25638, -0.25636, -0.25636, -0.25639, -0.25639,
        -0.25640]) tensor(-0.25637)
1e-05 0 tensor(5.83280e-05)
tensor([-0.25635, -0.25636, -0.25638, -0.25636, -0.25636, -0.25639, -0.25639,
        -0.25640]) tensor(-0.25637)
1e-05 500 tensor(5.48030e-08)
ten

In [46]:
%reload_ext autoreload
%autoreload all

wf = tps.unified_tensor()
mts = tps.merged_tensors()
dts = {}
for key, val in mts.items():
    print(key, val.shape)
    temp = torch.einsum('ABCDe,abcde->AaBbCcDd', val.conj(), val).reshape((chi**2, chi**2, chi**2, chi**2))
    dts.update({key: temp})

# ctmrg = tp.ctmrg.QuantumSquareCTMRG(wfs=mts, rho=8)
# impure_tx = torch.rand((chi**2, chi**2, chi**2, chi**2)), torch.rand((chi**2, chi**2, chi**2, chi**2))

# SVD to MPO
u, s, v = tp.linalg.tsvd(ham_xyz, group_dims=((0, 2), (1, 3)), svd_dims=(0, 0))
ss = torch.sqrt(s).diag()
us = torch.einsum('Aa,abc->Abc', ss, u)
sv = torch.einsum('Aa,abc->Abc', ss, v)
ham_mpo = us, sv
print(ham_xyz.shape, us.shape, sv.shape)

c, cx = (0, 0), (1, 0)
impure_tx = [
        torch.einsum(
            'ABCDE,fEe,abcde->AaBbCfcDd',
            mts[c].conj(), ham_mpo[0], mts[c]).reshape([chi**2, chi**2, chi**2*4, chi**2]),
        torch.einsum(
            'ABCDE,fEe,abcde->AfaBbCcDd', 
            mts[cx].conj(), ham_mpo[1], mts[cx]).reshape([chi**2*4, chi**2, chi**2, chi**2]),
        ]
for t in impure_tx:
    print(t.shape)

print(wf.shape, wf[0].shape)

(0, 0) torch.Size([3, 3, 3, 3, 2])
(1, 0) torch.Size([3, 3, 3, 3, 2])
(0, 1) torch.Size([3, 3, 3, 3, 2])
(1, 1) torch.Size([3, 3, 3, 3, 2])
torch.Size([2, 2, 2, 2]) torch.Size([4, 2, 2]) torch.Size([4, 2, 2])
torch.Size([9, 9, 36, 9])
torch.Size([36, 9, 9, 9])
torch.Size([4, 3, 3, 3, 3, 2]) torch.Size([3, 3, 3, 3, 2])


In [ ]:
%reload_ext autoreload
%autoreload all

ctmrg = tp.ctmrg.ClassicalSquareCTMRG(ts=dts, rho=32)

for k in range(16):
    ctmrg.rg_mu()
    ctmrg.rg_md()
    ctmrg.rg_ml()
    ctmrg.rg_mr()

    val_x = ctmrg.measure_twobody(c=(0, 0), impure_ts=impure_tx, direction='x')
    print(k, val_x.item())
    # val_y = ctmrg.measure_twobody(c=(0, 0), impure_ts=impure_ts_y, direction='y')
    # print(k, 'x-y:', val_x.item()*2, val_y.item()*2)

In [67]:
%reload_ext autoreload
%autoreload all

ctmrg = tp.ctmrg.QuantumSquareCTMRG(wfs=mts, rho=18)

# m = ctmrg.measure_onebody(c=(0, 0), op=(xyz.sx @ xyz.sx))
e = ctmrg.measure_twobody(c=(0, 0), op=xyz.twobody_ham())
print(0, e.item())

for k in range(20):
    ctmrg.rg_ml()
    ctmrg.rg_mu()
    ctmrg.rg_mr()
    ctmrg.rg_md()

    # m = ctmrg.measure_onebody(c=(0, 0), op=(xyz.sx @ xyz.sx))
    es = []
    es.append(ctmrg.measure_twobody(c=(0, 0), op=xyz.twobody_ham()).item())
    es.append(ctmrg.measure_twobody(c=(1, 0), op=xyz.twobody_ham()).item())
    es.append(ctmrg.measure_twobody(c=(0, 1), op=xyz.twobody_ham()).item())
    es.append(ctmrg.measure_twobody(c=(1, 1), op=xyz.twobody_ham()).item())
    print(k+1, es, np.mean(es))

0 -0.36682081636062736
1 [-0.26044712799104547, -0.2604673473167914, -0.26045408358892613, -0.26047907238864554] -0.26046190782135215
2 [-0.2597860821637273, -0.25980639173210357, -0.2597946969825964, -0.259817346923001] -0.25980112945035705
3 [-0.2597950402055385, -0.2598151123883663, -0.2598036869859181, -0.25982651782809035] -0.2598100893519783
4 [-0.2597893261163266, -0.2598107751097077, -0.259800349795851, -0.259822114814708] -0.2598056414591483
5 [-0.25978806612381944, -0.259808405018211, -0.25979943977284053, -0.2598199433822267] -0.2598039635742744
6 [-0.2597970776154902, -0.2598175142010437, -0.2598088139107068, -0.25982863682283835] -0.25981301063751977


KeyboardInterrupt: 

In [80]:
D, chi = 6, 18

c = torch.rand(chi, chi)
eu, el = torch.rand(chi, chi, D, D), torch.rand(chi, chi, D, D)
dt = torch.rand(D, D, D, D, D, D, D, D)

In [85]:
import opt_einsum as oe

def opt_con(c, eu, el, dt):
    return oe.contract('eh,efBb,ghAa,AaBbCcDd->fCcgDd', c, eu, el, dt)

def naive_con(c, eu, el, dt):
    t0 = oe.contract('efBb,eh->fhBb', eu, c)
    t1 = oe.contract('AaBbCcDd,ghAa->gDdhBbCc', dt, el)
    return oe.contract('gDdhBbCc,fhBb->fCcgDd', t1, t0)

import time
start =time.time()
for i in range(0):
    r = opt_con(c, eu, el, dt)
end = time.time()
print(end-start)

info = oe.contract_path('eh,efBb,ghAa,AaBbCcDd->fCcgDd', c, eu, el, dt)
print(info[0])
print(info[1])

2.4080276489257812e-05
[(0, 1), (0, 2), (0, 1)]
  Complete contraction:  eh,efBb,ghAa,AaBbCcDd->fCcgDd
         Naive scaling:  12
     Optimized scaling:  10
      Naive FLOP count:  7.053e+11
  Optimized FLOP count:  1.104e+9
   Theoretical speedup:  6.389e+2
  Largest intermediate:  4.199e+5 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   5           GEMM          efBb,eh->fBbh            ghAa,AaBbCcDd,fBbh->fCcgDd
   7           TDOT      fBbh,ghAa->fBbgAa               AaBbCcDd,fBbgAa->fCcgDd
  10           TDOT fBbgAa,AaBbCcDd->fCcgDd                       fCcgDd->fCcgDd
